In [1]:
from rich import print
from dotenv import load_dotenv
import os
from youtube_transcript_api import YouTubeTranscriptApi
from typing import List, Dict, Any, Generator
from langchain.schema import Document
import googleapiclient.discovery


load_dotenv()
%load_ext rich

In [2]:
youtube = googleapiclient.discovery.build(
    "youtube", "v3", developerKey=os.getenv("YOUTUBE_API_KEY")
)

In [3]:
search_result = (
    youtube.search()
    .list(
        part="snippet",
        q="how to make a website",
        maxResults=15,
        type="video",
        videoCaption="closedCaption",
    )
    .execute()
)

In [9]:
search_result


{
    'kind': 'youtube#searchListResponse',
    'etag': 'nbXPlK3wRdR2RRmb0bHnm9GlByc',
    'nextPageToken': 'CA8QAA',
    'regionCode': 'CA',
    'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 15},
    'items': [
        {
            'kind': 'youtube#searchResult',
            'etag': 'Kcl7CHFEZPjkijGPOj8ZTFwyw8o',
            'id': {'kind': 'youtube#video', 'videoId': 'YWA-xbsJrVg'},
            'snippet': {
                'publishedAt': '2018-01-03T15:32:33Z',
                'channelId': 'UCpWT_QfKk7BJIpn709YgsYA',
                'title': 'How to Make a Website in 10 mins - Simple &amp; Easy',
                'description': 'Start here ➜ https://visit.websitelearners.com/10minWeb18 ⏰️ Save Time, Let Ai Write Your Website Content ...',
                'thumbnails': {
                    'default': {
                        'url': 'https://i.ytimg.com/vi/YWA-xbsJrVg/default.jpg',
                        'width': 120,
                        'height': 90
                  

In [4]:
class YouTubeVideo:
    def __init__(
        self,
        video_id,
        title,
        description,
        published_at,
        channel_title,
        chunk_time_limit=120,
    ):
        self.video_id = video_id
        self.title = title
        self.description = description
        self.published_at = published_at
        self.channel_title = channel_title
        self.chunk_time_limit = chunk_time_limit

    def __repr__(self):
        return f"{self.title} by {self.channel_title} - {self.video_id}"

    def _get_transcript(self):
        try:
            transcript_list = YouTubeTranscriptApi.list_transcripts(self.video_id)
            for t in transcript_list:
                if t.language_code in ["en", "en-US"]:
                    return t.fetch()
                elif any(
                    lang["language_code"] == "en" for lang in t.translation_languages
                ):
                    return t.translate("en").fetch()
        except Exception as e:
            print(f"Error fetching transcript: {e} for video {self.video_id}")
        return None

    def _make_chunk_document(
        self, chunk_pieces: List[Dict], chunk_start_seconds: int
    ) -> Document:
        """Create Document from chunk of transcript pieces."""
        m, s = divmod(chunk_start_seconds, 60)
        h, m = divmod(m, 60)
        return Document(
            page_content=" ".join(
                map(lambda chunk_piece: chunk_piece["text"].strip(" "), chunk_pieces)
            ),
            metadata={
                "start_seconds": chunk_start_seconds,
                "start_timestamp": f"{h:02d}:{m:02d}:{s:02d}",
                "source": f"https://www.youtube.com/watch?v={self.video_id}&t={chunk_start_seconds}s",
                "title": self.title,
                "video_id": self.video_id,
            },
        )

    def _get_transcript_chunks(
        self, transcript_pieces: List[Dict]
    ) -> Generator[Document, None, None]:
        chunk_pieces: List[Dict[str, Any]] = []
        chunk_start_seconds = 0
        chunk_time_limit = self.chunk_time_limit
        for transcript_piece in transcript_pieces:
            piece_end = transcript_piece["start"] + transcript_piece["duration"]
            if piece_end > chunk_time_limit:
                if chunk_pieces:
                    yield self._make_chunk_document(chunk_pieces, chunk_start_seconds)
                chunk_pieces = []
                chunk_start_seconds = chunk_time_limit
                chunk_time_limit += self.chunk_time_limit

            chunk_pieces.append(transcript_piece)

        if len(chunk_pieces) > 0:
            yield self._make_chunk_document(chunk_pieces, chunk_start_seconds)

    def load(self):
        try:
            transcripts = self._get_transcript()
            if transcripts:
                return list(self._get_transcript_chunks(transcripts))
        except Exception as e:
            print(f"Error: {e} for video {self.video_id}")

        return []

In [5]:
items = search_result["items"]
yt_videos = [
    YouTubeVideo(
        video_id=item["id"]["videoId"],
        title=item["snippet"]["title"],
        description=item["snippet"]["description"],
        published_at=item["snippet"]["publishedAt"],
        channel_title=item["snippet"]["channelTitle"],
        chunk_time_limit=30,
    )
    for item in items
]

In [7]:
transcripts = [v.load() for v in yt_videos]
transcripts



[
    [
        Document(
            metadata={
                'start_seconds': 0,
                'start_timestamp': '00:00:00',
                'source': 'https://www.youtube.com/watch?v=YWA-xbsJrVg&t=0s',
                'title': 'How to Make a Website in 10 mins - Simple &amp; Easy',
                'video_id': 'YWA-xbsJrVg'
            },
            page_content="Hi guys, I'm Subhang from WebsiteLearners.com and in this video I'm\ngoing to show you how you can quickly make a\nwebsite. (in just 10 mins) Now after watching this video,\nyou will be able to make any kind of website just like\nthis, by using drag & drop. So, don't miss this video out and watch it till the end, to learn how to do it. Okay! So, I'm Subhang from\nWebsiteLearners.com and Let's\nstart making this website! Okay! So before we start, you need\nto First, Click the link, below this video. So I'm going to click this link."
        ),
        Document(
            metadata={
                'start_seconds': 30

## LangGraph

In [ ]:
from langchain_openai import ChatOpenAI

